# Modelo de Programación Lineal para el problema de Transporte
## 1. Planteamiento general del problema de transporte

$ \displaystyle{\text{Min } C = \sum_{i \in I} \sum_{j\in J} x_{ij} c_{ij}} $ <br>

donde:

* $x_{ij} = $ número de productos producidos en $i$ enviados a $j$
* $c_{ij} = $ costo de envío unitatio de $i$ a $j$
* $I = \{0,1,2 ...\} = $ conjunto de plantas de producción
* $J = \{0,1,2 ...\} = $ conjunto de supermercados o plantas receptoras

Además, se deben cumplir las siguientes restricciones:
1. **Restricción de oferta:** la suma de los productos enviados desde cada planta de producción a todos los supermercados debe ser menor o igual que la capacidad de producción $s_{i}$, es decir,

$\qquad \displaystyle {\sum_{j \in J} x_{ij} \leq s_i, \quad \forall \quad i \in I}$

2. **Restricción de demanda:** la suma de los productos enviados a cada supermercado por todos los centros de producción debe ser mayor o igual a la demanda  $d_{j}$, es decir,

$\qquad \displaystyle {\sum_{i \in I} x_{ij} \geq d_j, \quad \forall \quad j \in J}$


## 2. Ejemplo
Colum es un fabricante de yogur. La compañía tiene instalaciones de producción en Osorno, Valdivia y Puerto Fuy, que pueden producir 10,000 yogurts por semana. Colum distribuye su pedido en sus 3 instalaciones ubicadas en Puerto Montt, Temuco y Villarica. Colum quiere minimizar sus costos de transporte mientras satisface el pedido. Los costos de transporte entre cada instalación, la capacidad de producción y la demanda de cada destinos están en las tablas a continuación.

#### Capacidad de Producción por Plantas ($s_{i}$)

|    | Osorno | Valcivia | Puerto Fuy|
|----| ----| ----| ---- |
|$s_i$| 10,000 | 10,000 | 10,000 |

#### Demanda por Supermedado ($d_{j}$)

|    |Puerto Montt|Temuco|Villarica|
|----| ----| ----| ---- |
|$d_j$|11,000 |6,300|3,400 |

#### Costo de tranporte ($c_{ij}$)

|    |Puerto Montt|Temuco|Villarica|
|----| ----| ----| ---- |
|Osorno|1.04|1.27|1.22|
|Valcivia|1.23|1.93|0.6|
|Puerto Fuy|1.92|0.94|1.03|

## 3. Modelado del probIema
### 3.1 Indexado de las variables

In [61]:
plantas = [i for i in range(3)]
destinos = [j for j in range(3)]
arcos = [(i,j) for i in plantas for j in destinos]

arcos

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [62]:
capacidad = {0:10000,1:10000,2:10000}
demanda = {0:11000,1:6300,2:7400}
costo = {(0, 0):1.04, (0, 1):1.27, (0, 2):1.22, (1, 0):1.23, (1, 1):1.93, (1, 2):0.6, (2, 0):1.92, (2, 1):0.94, (2, 2):1.03}

### 3.2 CPLEX

In [63]:
from docplex.mp.model import Model

In [64]:
mdl = Model('problema_transporte')

Creación de variables:

In [65]:
x = mdl.integer_var_dict(arcos, name = 'x')

Función objetivo: $ \displaystyle{\text{Min } C = \sum_{i \in I} \sum_{j\in J} x_{ij} c_{ij}} $ <br>

In [66]:
mdl.minimize(mdl.sum(x[i]*costo[i] for i in arcos))

Restricciones:

1. **Restricción de oferta:** $\displaystyle {\sum_{j \in J} x_{ij} \leq s_i, \quad \forall \quad i \in I}$

In [67]:
for i in plantas:
    mdl.add_constraint(mdl.sum(x[i,j] for j in destinos) <= capacidad[i])

2. **Restricción de demanda:** $\displaystyle {\sum_{i \in I} x_{ij} \geq d_j, \quad \forall \quad j \in J}$

In [68]:
for j in destinos:
    mdl.add_constraint(mdl.sum(x[i,j] for i in destinos) >= demanda[j])

Impresión del modelo de optimización

In [69]:
print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: problema_transporte

Minimize
 obj: 1.040000000000 x_0_0 + 1.270000000000 x_0_1 + 1.220000000000 x_0_2
      + 1.230000000000 x_1_0 + 1.930000000000 x_1_1 + 0.600000000000 x_1_2
      + 1.920000000000 x_2_0 + 0.940000000000 x_2_1 + 1.030000000000 x_2_2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 <= 10000
 c2: x_1_0 + x_1_1 + x_1_2 <= 10000
 c3: x_2_0 + x_2_1 + x_2_2 <= 10000
 c4: x_0_0 + x_1_0 + x_2_0 >= 11000
 c5: x_0_1 + x_1_1 + x_2_1 >= 6300
 c6: x_0_2 + x_1_2 + x_2_2 >= 7400

Bounds

Generals
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



In [70]:
solucion = mdl.solve(log_output = True)

Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 30250.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 6 rows, 9 columns, and 18 nonzeros.
Reduced MIP has 0 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 6 rows, 9 columns, and 18 nonzeros.
Reduced MIP has 0 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        30250.0000        0.0000           100.00%
*     0+    0                        30245.7400   

In [71]:
mdl.get_solve_status()

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [72]:
solucion.display()

solution for: problema_transporte
objective: 21992.000
status: OPTIMAL_SOLUTION(2)
x_0_0 = 10000
x_1_0 = 1000
x_1_2 = 7400
x_2_1 = 6300
